Strategy
========
The Strategy class is an Abstract Base Class, so we will use ExampleStrategy.

In [1]:
from collections import namedtuple

import pandas as pd
import database.symbol as symboldb
import data as datalib
import puma as tw
from tomahawk import strategy

In [2]:
# Setup the market data
test_login = credentials('test', 'localhost')
db_credentials = credentials('test', 'localhost', prefix='db_')
seng = symboldb.symbol_engine('stock', **test_login)
symboldf = datalib.SymbolDBDataFeed({'stock': seng}, source='test_source_02')
hdm = datalib.HistoricalDataManager(symboldf, **db_credentials)
ldm = datalib.HistoricalDataManager(symboldf, **db_credentials)
mdm = datalib.MarketDataManager(hdm, ldm)

Initialization
--------------
At the initialization the Strategy object will attach itself to the other objects that are needed like OrderManager, Portfolio and PositionManager.

In [3]:
# Initialize the environment objects
oms = tw.OrderManager('test_unit', None)
pm = tw.PositionManager('test_unit', oms, None)
port = tw.Portfolio('test_port', oms, pm)

In [4]:
# The strategy needs an object that when called with dot-notation will return the environment objects
obj_bridge = namedtuple('OB', 'order_manager, portfolio, position_manager, market_data_manager')(oms, port, pm, mdm)

In [5]:
# Now intialize the strategy

strat = strategy.ExampleStrategy('strat-id', obj_bridge)

In [6]:
# Attach to a portfolio
port.add_strategy(strat)

In [7]:
# add symbols to the strategy. The strategy can access any information in the MarketDataManager, but adding the symbols here
# will cause them to be added to the MarketDataManager.

strat.add_symbols([('stock', 'MSFT', '1min'), ('stock', 'AAPL', '1min'), ('stock', 'test.sym.3', '1min')])

strat.symbol_tuples

In [8]:
# can also request the product_types that were added and the frequencies

print(strat.product_types)
print(strat.frequencies)
print(strat.symbols)

In [9]:
# add parameters as a dict

params = {'param1': 99, 'param2': [1,2,3], 'param3': 'BUY'}
strat.set_parameters(params)
print(strat.parameters)

Order methods
-------------

Create Order
------------

In [10]:
# create an order
order_id = strat.order('stock', 'test.sym.3', 'buy', 100, 'LIMIT', price=99.9)

oms.open_orders_df().show()

Cancel Order
------------

In [11]:
# cancel the order
strat.cancel_order(oms.order(order_id))

oms.open_orders_df().show()

Replace Order
-------------

In [12]:
order_id = strat.order('stock', 'test.sym.3', 'buy', 100, 'LIMIT', price=99.9)
order_obj = strat.get_order(order_id)
strat.replace_order(order_obj, 50, price=44.5)

oms.open_orders_df().show()

Get Order
---------

In [13]:
# Given a UUID, retreive the Order object
strat.get_order(order_id)

Intents
-------

In [14]:
# Create an intent and retreive the intents
strat.intent('stock', 'test.sym.3', 50)
strat.get_intent('stock', 'test.sym.3')

Position Information
--------------------

In [15]:
# enter a trade in the position manager
pm.enter_trade('orig-id', 'strat-id', pd.Timestamp('2010-01-01 09:31:00', tz='America/New_York'), 'stock', 'test.sym.3', 'buy', 250, 95.5)

In [16]:
# get the current position
strat.position('stock', 'test.sym.3')

on_X methods
------------
The Strategy object is event driven. The on_X methods will be invoked by the EventLooper. Execution of the Strategy will be
single threaded so that the control is not returned to the EventLoop until the on_X method is complete.

In [17]:
# on_start
# the examples strategy sets barcounr to 0 on start
strat.on_start()
strat.barcount

In [18]:
# on_begin_of_day
# the exmple appends each new day to a list
strat.on_begin_of_day(pd.Timestamp('2010-01-01 09:30:00', tz='America/New_York'))
strat.new_days

In [19]:
# on_market_open, example appends the datetime to a list
strat.on_market_open(pd.Timestamp('2010-01-01 09:30:00', tz='America/New_York'))
strat.open_days

In [20]:
# on_bar
# the ExampleStrategy will create some orders when called for this time
strat.on_bar(pd.Timestamp('2010-01-01 09:30:00', tz='America/New_York'))

oms.orders_df().show()

In [21]:
# on_fills
# when the symbol MSFT is filled this method is called and new orders are placed

ordr = oms.orders_list({'symbol': 'MSFT'})[0]
oms.change_state(ordr, 'FILLED')
ordr.add_fill(100, pd.Timestamp('2016-08-01 11:13:14', tz='America/New_York'), pd.Timestamp('2016-01-01 09:30:00', tz='America/New_York'), 50, 44.5, -0.5)
strat.on_fills(pd.Timestamp('2010-01-01 09:31:00'), [ordr])

# see the new order in the open orders
oms.open_orders_df().show()

In [22]:
# on_cancels
# running on_cancels will cause a new test.sym.3 to be created
strat.on_cancels(pd.Timestamp('2099-01-01 10:00:00', tz='America/New_York'), None)
oms.open_orders_df().show()

In [23]:
# on_market_close
strat.on_market_close(pd.Timestamp('2010-01-01 16:00:00', tz='America/New_York'))
strat.closed_days

In [24]:
# on_end_of_day
strat.on_end_of_day(pd.Timestamp('2010-01-01 16:00:00', tz='America/New_York'))
strat.days_done

In [25]:
# on_stop
strat.on_stop(pd.Timestamp('2010-01-01 09:30:00', tz='America/New_York'))
strat.start_stop